# 도서 데이터를 통한 SVD(Latent Factor ver)
Kaggle의 도서 데이터를 바탕으로 유저에게 새로운 도서를 추천하는 추천 시스템을 만들어보고자 한다.

In [1]:
import pandas as pd

In [2]:
books = pd.read_csv('C:/Users/USER/Desktop/RecommenderSystem/book_practice/books.csv')
ratings = pd.read_csv('C:/Users/USER/Desktop/RecommenderSystem/book_practice/ratings.csv')
users = pd.read_csv('C:/Users/USER/Desktop/RecommenderSystem/book_practice/users.csv')

C:\Users\USER\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
users

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN
...,...,...,...
278853,278854,"portland, oregon, usa",NaN
278854,278855,"tacoma, washington, united kingdom",50.0
278855,278856,"brampton, ontario, canada",NaN
278856,278857,"knoxville, tennessee, usa",NaN


## Data_처리

In [4]:
books

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...
...,...,...,...,...,...,...,...,...
271355,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm),http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...
271356,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...
271357,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...
271358,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...


In [5]:
# isbn, 책 제목, 저자, 년도, 출판사 정보만 남긴다.
books = books.iloc[:,0:4]

In [6]:
# 컬럼명을 편하게 변경
books.columns = ['ISBN', 'title', 'author', 'pub_year']

In [7]:
books.isna().sum()

ISBN        0
title       0
author      1
pub_year    0
dtype: int64

In [8]:
users

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN
...,...,...,...
278853,278854,"portland, oregon, usa",NaN
278854,278855,"tacoma, washington, united kingdom",50.0
278855,278856,"brampton, ontario, canada",NaN
278856,278857,"knoxville, tennessee, usa",NaN


In [9]:
# user 나이에 Nan이 많이 보인다. 연령대별로 어떤 책을 좋아하는지도 볼 수 있으니 나이를 쓸 수 있을지를 확인해보자.
users.isna().sum()

User-ID          0
Location         0
Age         110762
dtype: int64

In [10]:
# location의 경우 마지막의 국가만 활용해보려고 한다.
def cut_loc(loc):
    loc_cut_list = list(loc.split(', '))
    return loc_cut_list[-1]

In [11]:
users['country'] = users['Location'].apply(lambda x : cut_loc(x))

In [12]:
users

,User-ID,Location,Age,country
0,1,"nyc, new york, usa",NaN,usa
1,2,"stockton, california, usa",18.0,usa
2,3,"moscow, yukon territory, russia",NaN,russia
3,4,"porto, v.n.gaia, portugal",17.0,portugal
4,5,"farnborough, hants, united kingdom",NaN,united kingdom
...,...,...,...,...
278853,278854,"portland, oregon, usa",NaN,usa
278854,278855,"tacoma, washington, united kingdom",50.0,united kingdom
278855,278856,"brampton, ontario, canada",NaN,canada
278856,278857,"knoxville, tennessee, usa",NaN,usa


In [13]:
users = users.drop('Location', axis = 1)
users.columns = ['user_id', 'age', 'country']

In [14]:
users.isna().sum()

user_id         0
age        110762
country         0
dtype: int64

In [15]:
users.dtypes

user_id      int64
age        float64
country     object
dtype: object

In [16]:
def age_to_gen(age):
    if age == 0:
        age = 'unknown'
    elif age < 10:
        age = 'under_10'
    elif age < 20:
        age = '10s'
    elif age < 30:
        age = '20s'
    elif age < 40:
        age = '30s'
    elif age < 50:
        age = '40s'
    elif age < 60:
        age = '50s'
    else:
        age = 'over_60'
  
    return age

In [17]:
users.age.fillna(0, inplace = True)
users['gen'] = users['age'].apply(lambda x : age_to_gen(x))

In [18]:
users

,user_id,age,country,gen
0,1,0.0,usa,unknown
1,2,18.0,usa,10s
2,3,0.0,russia,unknown
3,4,17.0,portugal,10s
4,5,0.0,united kingdom,unknown
...,...,...,...,...
278853,278854,0.0,usa,unknown
278854,278855,50.0,united kingdom,50s
278855,278856,0.0,canada,unknown
278856,278857,0.0,usa,unknown


In [19]:
ratings

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
...,...,...,...
1149775,276704,1563526298,9
1149776,276706,0679447156,0
1149777,276709,0515107662,10
1149778,276721,0590442449,10


In [20]:
ratings.columns = ['user_id', 'ISBN', 'score']

In [21]:
# Pandas의 오픈 issue로 인해 해당 내용은 진행 불가
# rating_table = ratings.pivot_table(index = ['user_id'], columns = ['ISBN'], values = ['score']
# https://github.com/pandas-dev/pandas/issues/10582

In [22]:
# 위의 문제 해결을 위해, ratings를 쪼개서 만들고자 한다.
# 비슷한 유저군들끼리 묶기 위해 gen별로 table을 만들어 진행한다.
ratings_with_gen = pd.merge(ratings, users[['user_id', 'gen']],
                           left_on='user_id', right_on='user_id', how = 'left')
ratings_with_gen

,user_id,ISBN,score,gen
0,276725,034545104X,0,unknown
1,276726,0155061224,5,unknown
2,276727,0446520802,0,10s
3,276729,052165615X,3,10s
4,276729,0521795028,6,10s
...,...,...,...,...
1149775,276704,1563526298,9,unknown
1149776,276706,0679447156,0,10s
1149777,276709,0515107662,10,30s
1149778,276721,0590442449,10,10s


연령대를 확인 후 연령대별 추천 도서를 만들어주고자 한다.
따라서 연령별로 데이터를 나누자.

In [23]:
ratings_under_10= ratings_with_gen.loc[ratings_with_gen.gen == 'under_10',:].reset_index(drop = True)
ratings_10s = ratings_with_gen.loc[ratings_with_gen.gen == '10s',:].reset_index(drop = True)
ratings_20s = ratings_with_gen.loc[ratings_with_gen.gen == '20s',:].reset_index(drop = True)
ratings_30s = ratings_with_gen.loc[ratings_with_gen.gen == '30s',:].reset_index(drop = True)
ratings_40s = ratings_with_gen.loc[ratings_with_gen.gen == '40s',:].reset_index(drop = True)
ratings_50s = ratings_with_gen.loc[ratings_with_gen.gen == '50s',:].reset_index(drop = True)
ratings_over_60 = ratings_with_gen.loc[ratings_with_gen.gen == 'over_60',:].reset_index(drop = True)

In [24]:
# rating_table_under_10 = ratings_under_10.pivot_table(index = ['user_id'], columns = ['ISBN'], values = ['score'])
# rating_table_10s = ratings_10s.pivot_table(index = ['user_id'], columns = ['ISBN'], values = ['score'])
# rating_table_20s = ratings_20s.pivot_table(index = ['user_id'], columns = ['ISBN'], values = ['score'])
# rating_table_30s = ratings_30s.pivot_table(index = ['user_id'], columns = ['ISBN'], values = ['score'])
# rating_table_40s = ratings_40s.pivot_table(index = ['user_id'], columns = ['ISBN'], values = ['score'])
# rating_table_50s = ratings_50s.pivot_table(index = ['user_id'], columns = ['ISBN'], values = ['score'])
# rating_table_over_60 = ratings_over_60.pivot_table(index = ['user_id'], columns = ['ISBN'], values = ['score'])

## 모델 스케치

모델의 스케치를 위해 20대를 예로 방법을 간단하게 확인해보자.

In [42]:
rating_table_20s = pd.pivot_table(ratings_20s, index = 'user_id', columns = 'ISBN', values = 'score').fillna(0)

In [43]:
rating_table_20s

ISBN,+0451197399,0 14 02.9640,0 907 062 008,0/330/25864/8,00000000,000000000,0000000000,00000000000,0000000000000,0000000010,...,Q552146269,ULLSTEINBUCHN,V16382000,VENAFRO001,VG3862004,X000000000,"\0432534220\""""","\0679751521""",`3502103682,Ô½crosoft
user_id,,,,,,,,,,,,,,,,,,,,,
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
56,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
79,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
92,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278747,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
278843,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
278844,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
# 평균 정렬
import numpy as np
matrix_20s = np.matrix(rating_table_20s)
user_mean = np.mean(matrix_20s, axis = 1)
matrix_20s_centered = matrix_20s - user_mean.reshape(-1, 1)

NameError: name 'user_mean_reshape' is not defined

In [48]:
matrix_20s_centered

matrix([[-5.48491192e-05, -5.48491192e-05, -5.48491192e-05, ...,
         -5.48491192e-05, -5.48491192e-05, -5.48491192e-05],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [-1.46264318e-04, -1.46264318e-04, -1.46264318e-04, ...,
         -1.46264318e-04, -1.46264318e-04, -1.46264318e-04],
        ...,
        [-1.73688878e-04, -1.73688878e-04, -1.73688878e-04, ...,
         -1.73688878e-04, -1.73688878e-04, -1.73688878e-04],
        [-7.31321590e-05, -7.31321590e-05, -7.31321590e-05, ...,
         -7.31321590e-05, -7.31321590e-05, -7.31321590e-05],
        [-8.22736788e-05, -8.22736788e-05, -8.22736788e-05, ...,
         -8.22736788e-05, -8.22736788e-05, -8.22736788e-05]])

In [50]:
# SVD 적용
from scipy.sparse.linalg import svds
U, sigma, V = svds(matrix_20s_centered, k = 9)

Latent Factor를 얻었다.
User에 대해서 유사한 정보를 찾기 위해 U 행렬을 확인하고자 한다.

In [53]:
len(U)

17611

In [65]:
#  best-k 가장 유사한 유저 찾기
def pearson_corr(v1, v2):
    cov = np.cov(v1, v2)[0, 1] # 공분산 행렬로 나오지 않게 설정한다.
    std1 = np.std(v1)
    std2 = np.std(v2)
    return cov / (std1 * std2)

In [80]:
max_corr = -99
max_idx = -1
for i in range(1, len(U)):
    cur_corr = pearson_corr(U[0], U[i])
    if max_corr < cur_corr:
        max_corr = cur_corr
        max_idx = i
print(max_idx)

355


In [86]:
# 355번째 유저를 확인하는 방법
sorted(ratings_20s.user_id.unique())[355]

4875

In [92]:
#id = 10인 유저가 읽지 않았는데 id = 4875가 읽은 책을 확인하기.
id_10_read = ratings_20s.loc[ratings_20s.user_id == 10,:].ISBN
id_4875_read = ratings_20s.loc[ratings_20s.user_id == 4875,:].ISBN

In [93]:
id_4875_read

5171    157586326X
Name: ISBN, dtype: object

In [94]:
books.loc[books.ISBN=='157586326X',:]

,ISBN,title,author,pub_year
5477,157586326X,Things a Computer Scientist Rarely Talks About,Donald E. Knuth,2003


In [95]:
id_10_read

747    1841721522
748    8477024456
Name: ISBN, dtype: object

In [97]:
books.loc[books.ISBN=='1841721522',:]

,ISBN,title,author,pub_year
21,1841721522,New Vegetarian: Bold and Beautiful Recipes for...,Celia Brooks Brown,2001


In [98]:
books.loc[books.ISBN=='8477024456',:]

,ISBN,title,author,pub_year
